### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

The text file comes in JSON format and here are some keys and their description. They are only the most important ones for this task.

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In the cell, I tried to provide some helper code for better understanding and clearer vision

-**HINT**- Those lines of code may be not helping at all with your task.

In [50]:
# I will try to retrieve one instance of the file in a list of dictionaries
import json
records = [json.loads(line) for line in open('usa.gov_click_data.json')]
# Print the first occurance
records[0]

{'a': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11',
 'c': 'US',
 'nk': 1,
 'tz': 'America/New_York',
 'gr': 'MA',
 'g': 'A6qOVH',
 'h': 'wfLQtf',
 'l': 'orofrog',
 'al': 'en-US,en;q=0.8',
 'hh': '1.usa.gov',
 'r': 'http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf',
 'u': 'http://www.ncbi.nlm.nih.gov/pubmed/22415991',
 't': 1333307030,
 'hc': 1333307037,
 'cy': 'Danvers',
 'll': [42.576698, -70.954903]}

## Required

Write a script can transform the JSON files to a DataFrame and commit each file to a sparete CSV file in the target directory and consider the following:

        

All CSV files must have the following columns
- web_browser
        The web browser that has requested the service
- operating_sys
        operating system that intiated this request
- from_url

        The main URL the user came from

    **note**:

    If the retrived URL was in a long format `http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf`

     make it appear in the file in a short format like this `www.facebook.com`
     
    
- to_url

       The same applied like `to_url`
   
- city

        The city from which the the request was sent
    
- longitude

        The longitude where the request was sent
- latitude

        The latitude where the request was sent

- time_zone
        
        The time zone that the city follow
        
- time_in

        Time when the request started
- time_out
        
        Time when the request is ended
        
        
**NOTE** :

Because that some instances of the file are incomplete, you may encouter some NaN values in your transforamtion. Make sure that the final dataframes have no NaNs at all.

In [51]:
import pandas as pd
import numpy as np
import user_agents

In [52]:
def Bitly_tansform(input_file_path,output_save_path):
    """
    This Function transform the json files of the data collected from users by Bitly .
    
    Args:
    - input_file_path: The path/name of json file needed to transform.
    - output_save_path: The path/name of csv cleaned file needed to be saved.
    
    Returns:
    - No returns but it save the file to the path you entered in the second parameter.
    """
    
    #Read the json file and saving it into a DataFrame
    df = pd.read_json(input_file_path, lines=True)
    
    #Transforming the user_agent column into two columns, one stores the web browser and the other stores the os used
    df['web_browser'] = df['a'].apply(lambda x:user_agents.parse(x).browser.family)
    df['operating_sys'] = df['a'].apply(lambda x:user_agents.parse(x).os.family)
    
    #Transforming the url columns by splitting it and handling the rows with direct.
    df['from_url'] = df['r'].apply(lambda x: x.split('/')[2] if x.startswith('http') else 'direct')
    df['to_url'] = df['u'].apply(lambda x: x.split('/')[2] if x.startswith('http') else 'direct')
    
    #City column dosen't need any transformation
    df['city'] = df['cy']
    
    #Transforming the longitude and latitude by turning the null values into an empty list then spliting the column into a
    #-new dataframe then concatenating the new dataframe to the old one. 
    df['ll'] = df['ll'].apply(lambda x: [] if np.all(pd.isnull(x)) else x)
    new_df = pd.DataFrame(df['ll'].to_list(),columns = ['longitude','latitude'])
    df = pd.concat([df,new_df], axis =1)
    
    #Transforming the time columns from a time stamp into a datetime format.
    df['time_in'] = pd.to_datetime(df['t'], unit='s', errors='coerce')
    df['time_out'] = pd.to_datetime(df['hc'], unit='s', errors='coerce')
    
    #Time column dosen't need any transformation.
    df['time_zone'] = df['tz']
    
    #Specifing the needed columns into a list then dropping all other columns.
    col_names = ['web_browser','operating_sys','from_url','to_url','city','longitude','latitude','time_zone','time_in', 'time_out']
    df = df.loc[:, col_names]
    
    #dropping all rows with null values
    df.dropna(inplace = True)
    
    #Saving the dataframe into a CSV file to the needed path/name entered in the second parameter
    df.to_csv(output_save_path,index=False)

In [53]:
Bitly_tansform("usa.gov_click_data.json","usa.gov_click_data.csv")
df = pd.read_csv("usa.gov_click_data.csv")
df

,web_browser,operating_sys,from_url,to_url,city,longitude,latitude,time_zone,time_in,time_out
0,Chrome,Windows,www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York,2012-04-01 19:03:50,2012-04-01 19:03:57
1,Other,Other,www.AwareMap.com,www.monroecounty.gov,Provo,40.218102,-111.613297,America/Denver,2012-03-16 18:40:49,2011-06-16 22:13:13
2,IE,Windows,t.co,boxer.senate.gov,Washington,38.900700,-77.043098,America/New_York,2012-04-02 22:50:30,2012-04-02 22:50:35
3,Safari,Mac OS X,direct,apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo,2012-04-04 02:37:10,2012-04-04 02:37:24
4,Chrome,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2012-04-05 06:23:50,2012-04-05 06:23:59
5,Chrome,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2012-04-06 10:10:30,2012-04-06 10:10:48
6,Chrome,Windows,plus.url.google.com,www.nasa.gov,Luban,51.116699,15.283300,Europe/Warsaw,2012-04-07 13:57:10,2012-04-07 13:57:20
7,Firefox,Windows,direct,www.nysdot.gov,Seattle,47.595100,-122.332603,America/Los_Angeles,2012-04-11 01:17:10,2012-04-11 01:17:20
8,Firefox,Mac OS X,t.co,oversight.house.gov,Washington,38.937599,-77.092796,America/New_York,2012-04-12 05:04:00,2012-04-12 05:04:03
